In [1]:
import os
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Document_classifier_with_GithubCICD_FASTAPI_AWS\\research'

In [2]:
os.chdir("../")
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Document_classifier_with_GithubCICD_FASTAPI_AWS'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list

In [9]:
from docClassify.constants import *
from docClassify.utils.common import read_yaml, create_directories, get_size

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES,
        )

        return data_validation_config

In [11]:
import os
from docClassify.logger import logger

In [14]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    
    def validate_all_files_exist(self)-> bool:
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","data"))

            for file in all_files:
                file_size = get_size(Path(os.path.join("artifacts","data_ingestion","data", file)))
                if (file_size == 0):
                    logger.error(f"File is empty: {file}")
                else:
                    logger.info(f"File: {file} has size: {file_size}") 
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [15]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2024-02-05 21:38:49,591: INFO: common: yaml file: src\docClassify\config\config.yaml loaded successfully]
[2024-02-05 21:38:49,598: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-05 21:38:49,601: INFO: common: created directory at: artifacts]
[2024-02-05 21:38:49,604: INFO: common: created directory at: artifacts/data_validation]
[2024-02-05 21:38:49,608: INFO: 736030282: File: balance sheet has size: ~ 4 KB]
[2024-02-05 21:38:49,612: INFO: 736030282: File: cashflow has size: ~ 4 KB]
[2024-02-05 21:38:49,617: INFO: 736030282: File: income statement has size: ~ 4 KB]
